In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trasacntionTable").getOrCreate()

In [2]:
from pyspark.sql import SparkSession
from IPython.display import display

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("TransactionApp") \
    .getOrCreate()

In [3]:
import json

# raw_df.filter(F.col("_corrupt_record").isNull()).count()
with open(r'C:/Users/ACER/Desktop/kaggle/mcc_codes.json') as f:
    json_read = json.loads(f.read())
    # print(json_read)
# array of dict k, v
array_of_dicts = [{"mcc_code": k, "category": v} for k, v in json_read.items()]
# print(array_of_dicts)
mcc_codes_df = spark.createDataFrame(array_of_dicts)
mcc_codes_df.createOrReplaceTempView("mcc_codes_exploded")

In [4]:
df1 = spark.read.csv('C:/Users/ACER/Desktop/kaggle/transactions_data.csv', header=True, inferSchema=True)
df1.createOrReplaceTempView("transactions")
df1.show(5)

+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|         use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00|Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57|Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00|Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00|Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|      848|   3915| $46.41|

In [5]:
display(spark.sql("""
SELECT id, 
       TO_TIMESTAMP(date, 'yyyy-MM-dd HH:mm:ss') AS date,
       client_id,
       merchant_id,
       CAST(amount AS DECIMAL(10, 2)) AS amount,
       use_chip
FROM transactions
LIMIT 10
""").toPandas())

,id,date,client_id,merchant_id,amount,use_chip
0,7475327,2010-01-01 00:01:00,1556,59935,None,Swipe Transaction
1,7475328,2010-01-01 00:02:00,561,67570,None,Swipe Transaction
2,7475329,2010-01-01 00:02:00,1129,27092,None,Swipe Transaction
3,7475331,2010-01-01 00:05:00,430,27092,None,Swipe Transaction
4,7475332,2010-01-01 00:06:00,848,13051,None,Swipe Transaction
5,7475333,2010-01-01 00:07:00,1807,20519,None,Swipe Transaction
6,7475334,2010-01-01 00:09:00,1556,59935,None,Swipe Transaction
7,7475335,2010-01-01 00:14:00,1684,39021,None,Online Transaction
8,7475336,2010-01-01 00:21:00,335,50292,None,Online Transaction
9,7475337,2010-01-01 00:21:00,351,3864,None,Swipe Transaction


In [ ]:
display(spark.sql("""
SELECT DISTINCT
    r.merchant_id,
    r.merchant_city,
    r.merchant_state,
    r.zip,
    m.category AS merchant_category
FROM
    transactions_data r
LEFT JOIN
    mcc_codes_exploded m
ON
    r.mcc = m.mcc_code
LIMIT 5;
""").toPandas())

Py4JJavaError: An error occurred while calling o62.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 35) (SUYOG-PC executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint

In [ ]:
spark.catalog.listTables()

[Table(name='mcc_codes_exploded', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='transactions', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [6]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: double (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)



In [7]:
mcc_codes_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- mcc_code: string (nullable = true)

